# Classification of pedestrian and bicycle crashes and near-miss events
Pedestrian and bicycle crashes (and near-miss) events at intersection are identified by an AI-based model trained using video collected from traffic camera. This study evaluates the effectiveness or reliability of different automatically generated surrogates for crash modeling.

The models considered in this study:
*   Logistic regression
*   Decision tree
*   Random forest
*   XGBoost


## Setup

In [ ]:
# install packages and functions
%%capture

import numpy as np
import pandas as pd

# Visualization packages
import matplotlib.pyplot as plt
import seaborn
!pip install scikit-plot
import scikitplot as skplt
!pip install dtreeviz
!apt-get install graphviz
!pip install shap
import shap
plt.style.use('ggplot')

# Data preprocessing and performance metric
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold
from sklearn.metrics import confusion_matrix,classification_report,\
                            precision_score, recall_score, f1_score, accuracy_score
from sklearn.compose import make_column_transformer

# Machine learning packages
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
!pip install -q xgboost==1.5.0
import xgboost

# Generalized linear model - Logistic regression
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Oversampling
from imblearn.over_sampling import SMOTENC
from collections import Counter

import warnings
warnings.filterwarnings('ignore', category=UserWarning)

#Hyperparameter tuning
!pip install -q bayesian-optimization
from bayes_opt import BayesianOptimization

seed = 42 #for reproducibility

# Data description
We use the data from Pennsylvania Department of Transportation (PennDOT). The data consists of variables related to pedestrian and bicycle crashes collected using an automated detection system. Although, the datasheet contains additional variables (Evasive action, VRU awareness) that are not collected by the automated system, we do not use them for modeling purposes.

In [ ]:
#upload the data from drive (Note: if loaded in the files, do not need to run this again)
from google.colab import files
uploaded = files.upload()

In [ ]:
# load the dataset
dataframe = pd.read_excel('PedBike_safety.xlsx',sheet_name='data grouped')
dataframe.rename(columns = lambda x: x.replace(' ', '_'), inplace=True)

crash_data = dataframe[['PET', 'Arrived_First', 'R1_Movement', 'R1_Type','R1_Conflict_Speed', 'R1_Median_Speed', 
                        'R2_Movement','R2_Type', 'R2_Conflict_Speed', 'R2_Median_Speed','VRU_Type', 
                        'FarsideNearside_VRU', 'VRU_Location', 'Vehicle_Signal', 
                        'VRU_Signal', 'Proximity', 'Confirmed_Conflict', 'Lighting', 'Weather']]


In [ ]:
# crash_data = crash_data.loc[crash_data['VRU_Type'] == "Pedestrian"]

DataFrame contains data from various locations in PA. We would be using data from all locations, however, to ignore a specific location, use the following command after loading the data:
```
dataframe[dataframe['Location'].str.contains("Atherton Blue course near-miss") == False]
```
Next, response variable ```Confirmed_Conflict``` is mapped to binary values of 1 and 0 for positive and negative classes respectively. Before performing the exploratory analysis and running models, check if ```NULL/NA``` values in all columns are zero by running the following command:
```
crash_data.isnull().sum()
```

In [ ]:
pd.options.mode.chained_assignment = None

confirmed_conflict = {'Yes': 1, 'No': 0}
crash_data['Confirmed_Conflict'] = crash_data['Confirmed_Conflict'].map(confirmed_conflict)

crash_data = crash_data.dropna()

## Summary statistics

In [ ]:
plt.style.use('default')
fig, ax = plt.subplots(figsize=(4, 3))
fig = seaborn.kdeplot(x ='R1_Median_Speed', data = crash_data, palette ='mako', 
                      shade=True, label='R1_Conflict_Speed', ax=ax).set(xlim=(0, max(crash_data['R1_Conflict_Speed'].max(),crash_data['R1_Median_Speed'].max()))) 
fig = seaborn.kdeplot(x ='R2_Median_Speed', data = crash_data, palette ='mako', shade=True, label='R2_Median_Speed', ax=ax)
ax.legend()
plt.xlabel('Speed')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
seaborn.countplot(x ='Arrived_First', data = crash_data, palette ='mako') # to add title include this> .set(title='Title of Plot') 
plt.show()

In [ ]:
plt.style.use('default')
fig, ax = plt.subplots(figsize=(4, 3))
fig = seaborn.kdeplot(x ='PET', data = crash_data, palette ='mako', shade=True, label='PET', ax=ax)
plt.show()

In [ ]:
plt.style.use('default')
fig, ax = plt.subplots(figsize=(4, 3))
fig = seaborn.boxplot(x ='PET', y = 'R1_Type', data = crash_data, palette ='mako', notch = True, ax=ax)
plt.show()

# Methodology
The dataset is randomly split into two identically distributed datasets ```train``` and ```test``` for training the models and evaluating performances respectively. Notice, the class-imbalance in the dataset, which often limits the model performances. Here, we would consider training models with the true (i.e. imbalanced) data and balanced data by artificially adding synthetic samples corresponding to minority class.

In [ ]:
train, test = train_test_split(crash_data, test_size=0.3, random_state=seed) # 70% training and 30% test

counter = Counter(train['Confirmed_Conflict'])
print(counter)

We use Synthetic Minority Oversampling Technique-NC (SMOTE-NC) - a variant of SMOTE algorithm that works on numerical and categotical datasets, to balance our dataset. SMOTE works by creating synthetic samples for the minority class by interpolating the feature space distribution of the minority class. Note, the oversampling is performed on the ```train``` dataset only. If you want to run, the analysis without the oversampling, then proceed to next section.

In [ ]:
#define variable types
cat_data = ['Arrived_First', 'R1_Movement','R1_Type','R2_Movement','R2_Type', 'VRU_Type', 'FarsideNearside_VRU',
            'VRU_Location', 'Vehicle_Signal', 'VRU_Signal', 'Proximity', 'Lighting', 'Weather']

numeric_data = ['PET', 'R1_Conflict_Speed', 'R1_Median_Speed', 'R2_Conflict_Speed', 'R2_Median_Speed']

In [ ]:
smote = SMOTENC(random_state=seed, categorical_features= [train.drop(['Confirmed_Conflict'],axis=1).columns.get_loc(col) for col in cat_data])
X_sm, y_sm = smote.fit_resample(train.drop(['Confirmed_Conflict'],axis=1), train['Confirmed_Conflict'])

train = pd.concat([X_sm, y_sm], axis=1) #balanced 

## Logistic Regression
We will fit a Logistic regression model using the Generalized Linear Model ```glm()``` with argument ```family=sm.families.Binomial```. 
We use ```formula``` to define predictor and response variables in the model. Re-run the model by dropping one or many insignifcant variables from the variables list.

In [ ]:
logit_train = train.drop(train[train['Arrived_First']=="Motorcycle"].index) #we might need to drop specific categories based on p-value
logit_test = test.drop(test[test['Arrived_First']=="Motorcycle"].index) 

formula = 'Confirmed_Conflict ~ PET + Arrived_First + R1_Movement + R1_Conflict_Speed +\
          R2_Conflict_Speed + Vehicle_Signal + VRU_Signal + Proximity + Weather' # re-run model with differernt variable combination

formula_ped = 'Confirmed_Conflict ~ PET + R1_Movement + R1_Conflict_Speed +  R1_Median_Speed+ R2_Median_Speed +\
          R2_Conflict_Speed + Vehicle_Signal + VRU_Signal + Proximity + Weather + FarsideNearside_VRU' # re-run model with differernt variable combination          

logitmodel = smf.glm(formula_ped, logit_train, family=sm.families.Binomial())

model_logit = logitmodel.fit()
print(model_logit.summary())

Once the model is trained, we predict class probabilities ```y_logit_proba``` on the ```test``` split of the data with arguments ```transform=True``` and ```linear=False```. Note, the class probabilities need to be converted to corresponding class labels using a suitable threshold. Typically, a threshold of 0.5 is chosen for binary classifications, but should ideally chosen a threshold that maximizes a specific evaluation metric - for example, F1-score, as we consider here.

In [ ]:
y_logit_proba = model_logit.predict(logit_test, transform=True, linear=False) #prob being in Class 1

f1 = []
values = np.linspace(0, 1, num=21)
for cutoff in values:
  predictions_nominal = [0 if x < cutoff  else 1 for x in y_logit_proba]
  f1.append(f1_score(logit_test['Confirmed_Conflict'], predictions_nominal, average="macro"))

f1_logit = f1[np.argmax(f1)]
thres_logit = values[np.argmax(f1)]
print("Optimum macro F1 score = {:0.2f}; Threshold = {:0.2f}".format(f1_logit, thres_logit))

Classification results for threshold of 0.5 can be evalauted by using ```thres_logit=0.5``` before running the next code chunk.

In [ ]:
ypred_logit = [0 if x < thres_logit else 1 for x in y_logit_proba]
print(classification_report(logit_test['Confirmed_Conflict'], ypred_logit))

In [ ]:
# F1 score versus threshold
plt.figure(figsize=(5, 4))
plt.plot(values, f1)
plt.plot(thres_logit,f1_logit,marker="o", markersize=5)
plt.xlabel('Probability threshold')
plt.ylabel('F1 score')
plt.title('Optimum F1 score - Logit')
plt.show()

In [ ]:
class_prob = np.stack(((1-y_logit_proba).values, y_logit_proba.values), axis=1) # create a 2 dimensional array with probabilities of 0 and 1


skplt.metrics.plot_roc(logit_test['Confirmed_Conflict'], class_prob, cmap='tab20c', plot_micro=False, figsize=(5,4), text_fontsize=10)
skplt.metrics.plot_precision_recall(logit_test['Confirmed_Conflict'], class_prob, cmap='tab20c', plot_micro=False, figsize=(5,4), text_fontsize=10)
plt.show()

cross-validation score check

In [ ]:
from sklearn.base import BaseEstimator, RegressorMixin

class statsmodel(BaseEstimator, RegressorMixin):
    def __init__(self, sm_class, formula):
        self.sm_class = sm_class
        self.formula = formula
        self.model = None
        self.result = None
 
    def fit(self,data,dummy):
        self.model = self.sm_class(self.formula,data)
        self.result = self.model.fit()
 
    def predict(self,X):
        temp = self.result.predict(X)
        temp = [ 0 if x < thres_logit else 1 for x in temp]
        return temp

In [ ]:
# create a model
logitmodel_CV = statsmodel(smf.glm, formula)
cv = StratifiedKFold(n_splits=3, random_state=seed, shuffle=True)

# Print cross val score on this model
print (cross_val_score(logitmodel_CV, logit_train, logit_train['Confirmed_Conflict'], cv=cv, scoring='f1_macro'))

## Machine learning models

In [ ]:
# Ordinal encoding
transform_oe = make_column_transformer((OrdinalEncoder(), cat_data), 
                                       (StandardScaler(), numeric_data), remainder='passthrough',
                                       verbose_feature_names_out= False)
train_oe = transform_oe.fit_transform(train)
train_oe = pd.DataFrame(train_oe, columns=cat_data + numeric_data + ['Confirmed_Conflict'])

test_oe = transform_oe.fit_transform(test)
test_oe = pd.DataFrame(test_oe, columns=cat_data + numeric_data + ['Confirmed_Conflict'])

# One-hot encoding
transform_ohe = make_column_transformer((OneHotEncoder(handle_unknown='ignore'), cat_data),
                                        (StandardScaler(), numeric_data), remainder='passthrough',
                                        verbose_feature_names_out= False)
transform_ohe.fit(train)
train_ohe = transform_ohe.transform(train)
train_ohe = pd.DataFrame(train_ohe, columns=transform_ohe.get_feature_names_out())

test_ohe = transform_ohe.transform(test)
test_ohe = pd.DataFrame(test_ohe, columns=transform_ohe.get_feature_names_out())

# Select appropriate encoded data for ML models - Ordinal or OneHot
train_encoded = train_ohe
test_encoded = test_ohe

In [ ]:
len(test_ohe.columns)

In [ ]:
def dt_optimization(max_depth, min_samples_leaf):
    cv_splits = 3
    return cross_val_score(
               DecisionTreeClassifier(                                                             
                   max_depth=int(max(max_depth,1)),
                   min_samples_leaf=int(max(min_samples_leaf,1)), 
                   random_state=seed,   
                   class_weight="balanced"),  
               X=train_encoded.drop(['Confirmed_Conflict'],axis=1), 
               y=train_encoded['Confirmed_Conflict'], 
               cv=cv_splits,
               scoring="roc_auc",
               n_jobs=-1).mean()


parameters = {"max_depth": (5, 150),
              "min_samples_leaf": (2, 10)}


BO_dt = BayesianOptimization(dt_optimization, parameters,verbose = 2, random_state=seed)
BO_dt.maximize(init_points = 5, n_iter = 25)

print("Best result: {}; f(x) = {}.".format(BO_dt.max["params"], BO_dt.max["target"]))

In [ ]:
opt_history = []
for iter in range(0,len(BO_dt.res)):
    opt_history.append({'target': BO_dt.res[iter]['target'],
               'max_depth': BO_dt.res[iter]['params']['max_depth'],
               'min_samples_leaf':  BO_dt.res[iter]['params']['min_samples_leaf']
        })

opt_history = pd.DataFrame(opt_history)

In [ ]:
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import matplotlib.tri as tri
import numpy as np

fig,ax2 = plt.subplots(figsize=(6, 5))

# define grid.
xi = np.linspace(min(parameters['max_depth']), max(parameters['max_depth']),100)
yi = np.linspace(min(parameters['min_samples_leaf']), max(parameters['min_samples_leaf']),100)
# grid the data.
zi = griddata((opt_history['max_depth'], opt_history['min_samples_leaf']), opt_history['target'],
              (xi[None,:], yi[:,None]), method='cubic')
# contour the gridded data, plotting dots at the randomly spaced data points.
CS = plt.contour(xi,yi,zi,15,linewidths=0.5,colors='k')
CS = plt.contourf(xi,yi,zi,15,cmap=plt.cm.Greys)
plt.colorbar() # draw colorbar

plt.scatter(opt_history['max_depth'], opt_history['min_samples_leaf'], marker='o',c='k',s=20)
plt.scatter(opt_history['max_depth'][opt_history['target'].idxmax()],
           opt_history['min_samples_leaf'][opt_history['target'].idxmax()], marker='o',c='red',s=40)


plt.xlim(parameters['max_depth'])
plt.ylim(parameters['min_samples_leaf'])
plt.title('Objective function')
plt.xlabel('Max depth')
plt.ylabel('Min samples leaf')
plt.show()

In [ ]:
model= DecisionTreeClassifier(
                       max_depth=int(BO_dt.max["params"]["max_depth"]),
                       min_samples_leaf=int(BO_dt.max["params"]["min_samples_leaf"]),
                       random_state=seed, 
                       class_weight="balanced")

model_dt = model.fit(train_encoded.drop(['Confirmed_Conflict'],axis=1), train_encoded['Confirmed_Conflict'])
y_dt_proba = model_dt.predict_proba(test_encoded.drop(['Confirmed_Conflict'],axis=1))

In [ ]:
f1 = []
values = np.linspace(0, 1, num=21)
for cutoff in values:
  predictions_nominal = [0 if x < cutoff  else 1 for x in y_dt_proba[:,1]] #indexing needed for Class 1
  f1.append(f1_score(test['Confirmed_Conflict'], predictions_nominal, average="macro"))

f1_dt = f1[np.argmax(f1)]
thres_dt = values[np.argmax(f1)]
print("Optimum macro F1 score = {:0.2f}; Threshold = {:0.2f}.".format(f1_dt, thres_dt))

In [ ]:
ypred_dt = [0 if x < 0.5 else 1 for x in y_dt_proba[:,1]]
print(classification_report(test['Confirmed_Conflict'], ypred_dt))

In [ ]:
# F1 score versus threshold
plt.figure(figsize=(5, 4))
plt.plot(values, f1)
plt.plot(thres_dt,f1_dt,marker="o", markersize=5)
plt.xlabel('Probability threshold')
plt.ylabel('F1 score')
plt.title('Optimum F1 score - Decision tree')
plt.show()

In [ ]:
class_prob = y_dt_proba # create a 2 dimensional array with probabilities of 0 and 1

skplt.metrics.plot_roc(test['Confirmed_Conflict'], class_prob, cmap='tab20c', plot_micro=False, figsize=(5,4), text_fontsize=10)
skplt.metrics.plot_precision_recall(test['Confirmed_Conflict'], class_prob, cmap='tab20c', plot_micro=False, figsize=(5,4), text_fontsize=10)
plt.show()

In [ ]:
from dtreeviz.trees import * # remember to load the package
import dtreeviz

from PIL import Image
import io


m = dtreeviz.model(model_dt, X_train, y_train,
                target_name="target",
                feature_names=X_train.columns)

v = m.view()
v
# v.save("fig.svg")

In [ ]:
# v.save("fig.svg")
index

In [ ]:
# index = y_train[y_train==1]
x = X_train.iloc[:550]
m.view(x=x)


## Random Forest

In [ ]:
def rfc_optimization(n_estimators, max_depth, min_samples_split):
    cv_splits = 3
    return cross_val_score(
               RandomForestClassifier(
                   n_estimators=int(max(n_estimators,0)),                                                               
                   max_depth=int(max(max_depth,1)),
                   min_samples_split=int(max(min_samples_split,2)), 
                   n_jobs=-1, 
                   random_state=seed,   
                   class_weight="balanced"),  
               X=train_encoded.drop(['Confirmed_Conflict'],axis=1), 
               y=train_encoded['Confirmed_Conflict'],  
               cv=cv_splits,
               scoring="roc_auc",
               n_jobs=-1).mean()


parameters = {"n_estimators": (10, 1000),
              "max_depth": (5, 150),
              "min_samples_split": (2, 10)}


BO_rf = BayesianOptimization(rfc_optimization, parameters, verbose = 2, random_state=seed)
BO_rf.maximize(init_points = 5, n_iter = 20)

print("Best result: {}; f(x) = {}.".format(BO_rf.max["params"], BO_rf.max["target"]))

In [ ]:
opt_history = []
for iter in range(0,len(BO_rf.res)):
    opt_history.append({'target': BO_rf.res[iter]['target'],
               'max_depth': BO_rf.res[iter]['params']['max_depth'],
               'n_estimators': BO_rf.res[iter]['params']['n_estimators'],
               'min_samples_split':  BO_rf.res[iter]['params']['min_samples_split']
        })

opt_history = pd.DataFrame(opt_history)

In [ ]:
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import matplotlib.tri as tri
import numpy as np

fig,ax2 = plt.subplots(figsize=(6, 5))

# define grid.
xi = np.linspace(min(parameters['max_depth']), max(parameters['max_depth']),100)
yi = np.linspace(min(parameters['min_samples_split']), max(parameters['min_samples_split']),100)
# grid the data.
zi = griddata((opt_history['max_depth'], opt_history['min_samples_split']), opt_history['target'],
              (xi[None,:], yi[:,None]), method='cubic')
# contour the gridded data, plotting dots at the randomly spaced data points.
CS = plt.contour(xi,yi,zi,15,linewidths=0.5,colors='k')
CS = plt.contourf(xi,yi,zi,15,cmap=plt.cm.Greys)
plt.colorbar() # draw colorbar

plt.scatter(opt_history['max_depth'], opt_history['min_samples_split'], marker='o',c='k',s=20)
plt.scatter(opt_history['max_depth'][opt_history['target'].idxmax()],
           opt_history['min_samples_split'][opt_history['target'].idxmax()], marker='o',c='red',s=30)


plt.xlim(parameters['max_depth'])
plt.ylim(parameters['min_samples_split'])
plt.title('Objective function')
plt.xlabel('Max depth')
plt.ylabel('Min sample split')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import matplotlib.tri as tri
import numpy as np

fig,ax2 = plt.subplots(figsize=(6, 5))

# define grid.
xi = np.linspace(min(parameters['max_depth']), max(parameters['max_depth']),100)
yi = np.linspace(min(parameters['n_estimators']), max(parameters['n_estimators']),100)
# grid the data.
zi = griddata((opt_history['max_depth'], opt_history['n_estimators']), opt_history['target'],
              (xi[None,:], yi[:,None]), method='cubic')
# contour the gridded data, plotting dots at the randomly spaced data points.
CS = plt.contour(xi,yi,zi,15,linewidths=0.5,colors='k')
CS = plt.contourf(xi,yi,zi,15,cmap=plt.cm.Greys)
plt.colorbar() # draw colorbar

plt.scatter(opt_history['max_depth'], opt_history['n_estimators'], marker='o',c='k',s=20)
plt.scatter(opt_history['max_depth'][opt_history['target'].idxmax()],
           opt_history['n_estimators'][opt_history['target'].idxmax()], marker='o',c='red',s=30)


plt.xlim(parameters['max_depth'])
plt.ylim(parameters['n_estimators'])
plt.title('Objective function')
plt.xlabel('Max depth')
plt.ylabel('No of estimators')
plt.show()


In [ ]:
model= RandomForestClassifier(
                       n_estimators=int(BO_rf.max["params"]["n_estimators"]),
                       max_depth=int(BO_rf.max["params"]["max_depth"]),
                       min_samples_split=int(BO_rf.max["params"]["min_samples_split"]),
                       n_jobs=-1, 
                      random_state=seed,   
                      class_weight="balanced")

model_rf = model.fit(train_encoded.drop(['Confirmed_Conflict'],axis=1), train_encoded['Confirmed_Conflict'])
y_rf_proba = model_rf.predict_proba(test_encoded.drop(['Confirmed_Conflict'],axis=1))

In [ ]:
f1 = []
values = np.linspace(0, 1, num=21)
for cutoff in values:
  predictions_nominal = [0 if x < cutoff  else 1 for x in y_rf_proba[:,1]] #indexing needed for Class 1
  f1.append(f1_score(test['Confirmed_Conflict'], predictions_nominal, average="macro"))

f1_rf = f1[np.argmax(f1)]
thres_rf = values[np.argmax(f1)]
print("Optimum macro F1 score = {:0.2f}; Threshold = {:0.2f}.".format(f1_rf, thres_rf))

In [ ]:
ypred_rf = [0 if x < thres_rf else 1 for x in y_rf_proba[:,1]]
print(classification_report(test['Confirmed_Conflict'], ypred_rf))

In [ ]:
# F1 score versus threshold
plt.figure(figsize=(5, 4))
plt.plot(values, f1)
plt.plot(thres_rf,f1_rf,marker="o", markersize=5)
plt.xlabel('Probability threshold')
plt.ylabel('F1 score')
plt.title('Optimum F1 score - Random Forest')
plt.show()

In [ ]:
class_prob = y_rf_proba # create a 2 dimensional array with probabilities of 0 and 1

skplt.metrics.plot_roc(test['Confirmed_Conflict'], class_prob, cmap='tab20c', plot_micro=False, figsize=(5,4), text_fontsize=10)
skplt.metrics.plot_precision_recall(test['Confirmed_Conflict'], class_prob, cmap='tab20c', plot_micro=False, figsize=(5,4), text_fontsize=10)
plt.show()

## XGBoost

In [ ]:
def xgb_optimization(eta, gamma, max_depth):
            cv_splits = 3
            return cross_val_score(
                   xgboost.XGBClassifier(
                       objective="binary:logistic",
                       learning_rate=max(eta, 0),
                       gamma=max(gamma, 0),
                       max_depth=int(max_depth),                                               
                       seed=42,
                       nthread=-1,
                       scale_pos_weight = len(train_encoded['Confirmed_Conflict'][train_encoded['Confirmed_Conflict'] == 0])/
                                          len(train_encoded['Confirmed_Conflict'][train_encoded['Confirmed_Conflict'] == 1])),  
                   X=train_encoded.drop(['Confirmed_Conflict'],axis=1), 
                   y=train_encoded['Confirmed_Conflict'],
                   cv=cv_splits,
                   scoring="roc_auc",
                   n_jobs=-1).mean()

parameters = {"eta": (0.001, 0.4),
              "gamma": (0, 20),
              "max_depth": (1, 1000)}
    

BO_xg = BayesianOptimization(xgb_optimization, parameters,verbose = 2, random_state= seed)
BO_xg.maximize(init_points = 5, n_iter = 20)

print("Best result: {}; f(x) = {}.".format(BO_xg.max["params"], BO_xg.max["target"]))

In [ ]:
opt_history = []
for iter in range(0,len(BO_rf.res)):
    opt_history.append({'target': BO_xg.res[iter]['target'],
               'eta': BO_xg.res[iter]['params']['eta'],
               'gamma': BO_xg.res[iter]['params']['gamma'],
               'max_depth':  BO_xg.res[iter]['params']['max_depth']
        })

opt_history = pd.DataFrame(opt_history)

In [ ]:
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import matplotlib.tri as tri
import numpy as np

fig,ax2 = plt.subplots(figsize=(6, 5))

# define grid.
xi = np.linspace(min(parameters['eta']), max(parameters['eta']),100)
yi = np.linspace(min(parameters['gamma']), max(parameters['gamma']),100)
# grid the data.
zi = griddata((opt_history['eta'], opt_history['gamma']), opt_history['target'],
              (xi[None,:], yi[:,None]), method='cubic')
# contour the gridded data, plotting dots at the randomly spaced data points.
CS = plt.contour(xi,yi,zi,15,linewidths=0.5,colors='k')
CS = plt.contourf(xi,yi,zi,15,cmap=plt.cm.Greys)
plt.colorbar() # draw colorbar

plt.scatter(opt_history['eta'], opt_history['gamma'], marker='o',c='k',s=20)
plt.scatter(opt_history['eta'][opt_history['target'].idxmax()],
           opt_history['gamma'][opt_history['target'].idxmax()], marker='o',c='red',s=30)


plt.xlim(parameters['eta'])
plt.ylim(parameters['gamma'])
plt.title('Objective function')
plt.xlabel('Eta')
plt.ylabel('Gamma')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import matplotlib.tri as tri
import numpy as np

fig,ax2 = plt.subplots(figsize=(6, 5))

# define grid.
xi = np.linspace(min(parameters['eta']), max(parameters['eta']),100)
yi = np.linspace(min(parameters['max_depth']), max(parameters['max_depth']),100)
# grid the data.
zi = griddata((opt_history['eta'], opt_history['max_depth']), opt_history['target'],
              (xi[None,:], yi[:,None]), method='cubic')
# contour the gridded data, plotting dots at the randomly spaced data points.
CS = plt.contour(xi,yi,zi,15,linewidths=0.5,colors='k')
CS = plt.contourf(xi,yi,zi,15,cmap=plt.cm.Greys)
plt.colorbar() # draw colorbar

plt.scatter(opt_history['eta'], opt_history['max_depth'], marker='o',c='k',s=20)
plt.scatter(opt_history['eta'][opt_history['target'].idxmax()],
           opt_history['max_depth'][opt_history['target'].idxmax()], marker='o',c='red',s=30)


plt.xlim(parameters['eta'])
plt.ylim(parameters['max_depth'])
plt.title('Objective function')
plt.xlabel('Eta')
plt.ylabel('Max depth')
plt.show()

In [ ]:
model = xgboost.XGBClassifier(
                       objective="binary:logistic",
                       eta=(BO_xg.max["params"]["eta"]),
                       gamma=(BO_xg.max["params"]["gamma"]),
                       max_depth=int(BO_xg.max["params"]["max_depth"]),                                               
                       seed= seed,
                       nthread=-1,
                       scale_pos_weight = len(train_encoded['Confirmed_Conflict'][train_encoded['Confirmed_Conflict'] == 0])/
                                          len(train_encoded['Confirmed_Conflict'][train_encoded['Confirmed_Conflict'] == 1]))

model_xg = model.fit(train_encoded.drop(['Confirmed_Conflict'],axis=1), train_encoded['Confirmed_Conflict'])
y_xg_proba = model_xg.predict_proba(test_encoded.drop(['Confirmed_Conflict'],axis=1))

In [ ]:
f1 = []
values = np.linspace(0, 1, num=21)
for cutoff in values:
  predictions_nominal = [0 if x < cutoff  else 1 for x in y_xg_proba[:,1]] #indexing needed for Class 1
  f1.append(f1_score(test['Confirmed_Conflict'], predictions_nominal, average="macro"))

f1_xg = f1[np.argmax(f1)]
thres_xg = values[np.argmax(f1)]
print("Optimum macro F1 score = {:0.2f}; Threshold = {:0.2f}.".format(f1_xg, thres_xg))

In [ ]:
ypred_xg = [0 if x < 0.5 else 1 for x in y_xg_proba[:,1]]
print(classification_report(test['Confirmed_Conflict'], ypred_xg))

In [ ]:
# F1 score versus threshold
plt.figure(figsize=(5, 4))
plt.plot(values, f1)
plt.plot(thres_xg,f1_xg,marker="o", markersize=5)
plt.xlabel('Probability threshold')
plt.ylabel('F1 score')
plt.title('Optimum F1 score - XGBoost')
plt.show()

In [ ]:
class_prob = y_xg_proba # create a 2 dimensional array with probabilities of 0 and 1

skplt.metrics.plot_roc(test['Confirmed_Conflict'], class_prob, cmap='tab20c', plot_micro=False, figsize=(5,4), text_fontsize=10)
skplt.metrics.plot_precision_recall(test['Confirmed_Conflict'], class_prob, cmap='tab20c', plot_micro=False, figsize=(5,4), text_fontsize=10)
plt.show()

In [ ]:
feature_imp = pd.Series(model_rf.feature_importances_,
                        index=train_encoded.drop(['Confirmed_Conflict'],axis=1).columns).sort_values(ascending=False)

# Creating a bar plot
# fig, ax = plt.subplots(figsize=(6, 15))
seaborn.barplot(x=feature_imp, y=feature_imp.index, palette = "mako",)
# Add labels to your graph
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
plt.legend()
plt.show()

In [ ]:
shap_values = shap.Explainer(model_dt).shap_values(train_encoded.drop(['Confirmed_Conflict'],axis=1))
shap.summary_plot(shap_values[1], train_encoded.drop(['Confirmed_Conflict'],axis=1))
